#### Importando as bibliotecas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import scipy.stats
from scipy.stats import wilcoxon
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
%matplotlib inline

#### Abrindo os dados

In [4]:
#Abrindo dados de lentidao
arruamento_e_lentidao = pd.read_excel(r'D:\Trabalho\faculdade\TCC\gis\tabelas\arruamento_valido_e_lentidao.xlsx', usecols = ['data_medicao','corredor','Shape_Length','lentidao'])
arruamento_e_lentidao = arruamento_e_lentidao.rename(columns = {'Shape_Length':'tamanho_corredor'})
arruamento_e_lentidao['data_medicao'] = pd.to_datetime(arruamento_e_lentidao['data_medicao'])
#arruamento_e_lentidao = arruamento_e_lentidao.set_index(['data_medicao','corredor'])

In [5]:
#Abrindo dados de chuva
chuva = pd.read_excel(r'D:\Trabalho\faculdade\TCC\gis\tabelas\join_chuva.xlsx',usecols = ['data_medicao','corredor','precipitacao'])
chuva['data_medicao'] = pd.to_datetime(chuva['data_medicao'])
#chuva = chuva.set_index(['data_medicao','corredor'])

In [6]:
#Abrindo dados de alagamento
alagamentos = pd.read_excel(r'D:\Trabalho\faculdade\TCC\gis\tabelas\join_alagamentos.xlsx', usecols=['data_medicao','corredor','soma_duracoes_alagamentos','quantidade_alagamentos'])
alagamentos['data_medicao'] = pd.to_datetime(alagamentos['data_medicao'])
#alagamentos = alagamentos.set_index(['data_medicao','corredor'])

In [7]:
#Abrindo dados de acidentes
acidentes = pd.read_excel(r'D:\Trabalho\faculdade\TCC\gis\tabelas\join_acidentes.xlsx', usecols=['data_medicao','corredor','quantidade_acidentes'])
acidentes['data_medicao'] = pd.to_datetime(acidentes['data_medicao'])
#acidentes = acidentes.set_index(['data_medicao','corredor'])

In [8]:
#Abrindo dados socioeconômicos
socioeconomico = pd.read_excel(r'D:\Trabalho\faculdade\TCC\gis\tabelas\join_socioeconomico.xlsx').drop(['OBJECTID','Shape_Length'], axis=1)
socioeconomico['data_medicao'] = pd.to_datetime(socioeconomico['data_medicao'])
#acidentes = acidentes.set_index(['data_medicao','corredor'])

#### Juntando os dados

In [9]:
#Juntando os de chuva
merge_chuva = pd.merge(arruamento_e_lentidao,chuva,on=['data_medicao','corredor'], how='left')
merge_chuva = merge_chuva.fillna(0)

In [10]:
#Juntando dados de alagamento
merge_alagamentos = pd.merge(merge_chuva ,alagamentos,on=['data_medicao','corredor'],how='left')
merge_alagamentos = merge_alagamentos.fillna(0)

In [11]:
#Juntando dados de acidentes
merge_acidentes = pd.merge(merge_alagamentos ,acidentes,on=['data_medicao','corredor'],how='left')
merge_acidentes = merge_acidentes.fillna(0)

In [12]:
#Juntando dados socioeconomicos
merge_socioeconomico = pd.merge(merge_acidentes ,socioeconomico,on=['data_medicao','corredor'],how='left')
data = merge_socioeconomico.copy()

#### Adicionando campos com informações sobre o horário de medição da lentidão

In [13]:
data['dia'] = data['data_medicao'].dt.weekday.apply(lambda x: 'dia_de_semana' if x<4 else 'fim _de_semana')


def get_daytime(x):
    if x in range(0,7):
        return 'madrugada'
    elif x in range(7,13):
        return 'manha'
    elif x in range(13,19):
        return 'tarde'
    elif x in range(19,24):
        return 'noite'
        
data['periodo_do_dia'] = data['data_medicao'].dt.hour.apply(lambda x: get_daytime(x))

def get_season(x):
    
    spring = range(80, 172)
    summer = range(172, 264)
    fall = range(264, 355)
    # winter = everything else

    if x in spring:
        return'primavera'
    elif x in summer:
        return'verao'
    elif x in fall:
        return 'outono'
    else:
        return 'inverno'
    
    
data['estacao'] = data['data_medicao'].dt.dayofyear.apply(lambda x: get_season(x))

In [14]:
one_hot_encoded_columns = pd.get_dummies(data[['dia','periodo_do_dia','estacao']])
data = data.drop(['dia','periodo_do_dia','estacao'], axis=1)
data = data.join(one_hot_encoded_columns)
#data = data.set_index(['data_medicao','corredor'])
out_path = r'D:\Trabalho\faculdade\TCC\outputs\tabelas\dados_treino.csv'
data.to_csv(out_path, index=False)